# Convolution in GPU 
## working , but not faster
## for small system, exploit GPU is difficult

In [2]:
import numpy as np
from scipy import signal
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

from timeit import default_timer as timer

#speed improve, useless if parrallel activate
from numba import jit,autojit

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


In [25]:
N = 1024
L = 32
dx = np.float(L)/N
Nw = 8
Lw = int(np.floor(8/dx)+1)
LLw = int(np.floor(8/dx)/2)
Lw*dx

8.03125

In [26]:
"""initial w"""
def initial_kernel():
    
    np.random.seed(0)
    w = np.random.normal(0,1,Nw*Lw)*0.02
    w=w.reshape((Nw,Lw))

    return w

In [27]:
w = initial_kernel()


In [136]:
def conv_cpu (rho,w):
    rho_pad=np.zeros((N+Lw-1))
    rho_pad[LLw:LLw+N]=rho
    rho_pad[0:LLw]=rho_pad[N:N+LLw]
    rho_pad[N+LLw:N+LLw*2]=rho_pad[LLw:LLw+LLw]
    n = np.correlate(rho_pad,w)
    return n*dx

@jit(cache=True,nopython=True)
def conv_cpu2 (rho,w):
    n=np.zeros(N)
    rho_pad=np.zeros((N+Lw-1))
    rho_pad[LLw:LLw+N]=rho
    rho_pad[0:LLw]=rho_pad[N:N+LLw]
    rho_pad[N+LLw:N+LLw*2]=rho_pad[LLw:LLw+LLw]
    for i in range (N):
        n[i]=np.sum(w*rho_pad[i:i+Lw])
    return n*dx



@jit(cache=True)
def conv_fft (rho,w):
    rho_pad=np.zeros((N+Lw-1))
    rho_pad[LLw:LLw+N]=rho
    rho_pad[0:LLw]=rho_pad[N:N+LLw]
    rho_pad[N+LLw:N+LLw*2]=rho_pad[LLw:LLw+LLw]
    n = signal.correlate(rho_pad,w,mode='valid')
    
    return n*dx

mod = SourceModule("""
  __global__ void conv_cuda(float *n,float *rho, float *w,int Lw,int N)
  {
    int idx = threadIdx.x+blockIdx.x*blockDim.x;
    n[idx]=0;
    int LLw = (Lw-1)/2;
    for(int i=0;i<Lw;i++)
    {
        int p = (idx-LLw+i)%N;
        
        if(p<0) p += N;
        
        if(p>=N) p -= N;
        
        n[idx]+=rho[p]*w[i];
        //n[idx]=N;
    }
  }
  """)

conv_cuda2 = mod.get_function("conv_cuda")
def conv(rho,w):
    n = np.zeros(N,dtype='single')
    rho = rho.astype('single')
    w=w.astype('single')
    #print(type(w))
    conv_cuda2(cuda.Out(n),cuda.In(rho),cuda.In(w),np.int32(Lw),np.int32(N), block=(512,1,1),grid=(int((N+511)/512),1,1))
    return n*dx

In [137]:
rho = np.random.normal(1,1,N)
#conv(rho,w[0])
#conv_fft(rho,w[0])
np.sum((conv_cpu(rho,w[0])-conv_cpu2(rho,w[0]))**2)


7.631810967752504e-32

In [138]:
rho = np.full(N,1)
np.sum((conv_fft(rho,w[0])-conv(rho.astype('single'),w[0].astype('single')))**2)

3.9016399075754114e-16

In [139]:
%timeit -n100 conv_fft(rho,w[0])

84.5 µs ± 3.57 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [140]:
%timeit -n100 conv(rho,w[0])

640 µs ± 31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [141]:
#rho=rho.astype('single')
#w0 = w[0].astype('single')
%timeit -n100 conv_cpu(rho,w[0])

52.7 µs ± 6.63 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [143]:
%timeit -n1 conv_cpu2(rho,w[0])

529 µs ± 49.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
